In [ ]:
# Goal: Make a multiprocessing pool, ensure each worker has persistent state.
import multiprocessing as mp
import time
import random

In [ ]:
STOP = 'STOP'

# From: "An example showing how to use queues to feed tasks to a collection of worker processes and collect the results:"
def worker(inputs, outputs):
    count = 0
    for value in iter(inputs.get, STOP):
        time.sleep(random.uniform(0.05, 0.1))
        count += 1
        outputs.put((count, value, mp.current_process().name))
    print("DONE")

inputs = mp.Queue()
map(inputs.put, range(100))
outputs = mp.Queue()

ps = []
for i in range(10):
    p = mp.Process(target=worker, args=(inputs, outputs))
    ps.append(p)
    p.start()
for p in ps:
    inputs.put(STOP)
for p in ps:
    p.join()

while not outputs.empty():
    print(outputs.get())
assert outputs.empty()

In [ ]:
from multiprocessing import current_process
import random
import time

def worker(values):
    # Count how much work each worker does.
    count = 0
    for value in values:
        time.sleep(random.uniform(1.0, 1.1))
        count += 1
        yield (current_process().name, count, value)

output_iter = parallel_generator_imap_unordered(
    worker, range(5), process_count=3)
list(output_iter)